In [1]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2017 = db.rm_patent_cn_2017
    db_rm_us_2017 = db.rm_patent_us_2017
    db_rm_de_2017 = db.rm_patent_de_2017

    
except errors.ConnectionFailure as err:
    print(err)

# 2017

## Company數量

In [6]:
data_assignee = db_rm_de_2017.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2017.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2017.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
2019,Samsung Electronics Co Ltd,678
1876,Qualcomm Inc,655
2272,Shenzhen Shenglu IoT Communication Technology ...,622
1250,Intel Corp,403
2546,Telefonaktiebolaget LM Ericsson AB,395
1155,Huawei Technologies Co Ltd,291
1262,International Business Machines Corp,249
2865,Xian Irain IoT Technology Service Co Ltd,139
1562,Microsoft Technology Licensing LLC,132
518,Cisco Technology Inc,131


In [43]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [44]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Samsung Electronics Co Ltd,678,6.519147
1,Qualcomm Inc,655,6.484635
2,Shenzhen Shenglu IoT Communication Technology ...,622,6.432940
3,Intel Corp,403,5.998937
4,Telefonaktiebolaget LM Ericsson AB,395,5.978886
5,Huawei Technologies Co Ltd,291,5.673323
6,International Business Machines Corp,249,5.517453
7,Xian Irain IoT Technology Service Co Ltd,139,4.934474
8,Microsoft Technology Licensing LLC,132,4.882802
9,Cisco Technology Inc,131,4.875197


公司名稱正規化

In [45]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba"))

In [46]:
com_df

,company,count,count_log
0,Samsung Electronics,678,6.519147
1,Qualcomm,655,6.484635
2,Shenzhen Shenglu IoT Communication Technology,622,6.432940
3,Intel,403,5.998937
4,Telefonaktiebolaget LM Ericsson AB,395,5.978886
5,Huawei Technologies,291,5.673323
6,IBM,249,5.517453
7,Xian Irain IoT Technology Service,139,4.934474
8,Microsoft Technology Licensing LLC,132,4.882802
9,Cisco Technology,131,4.875197


In [47]:
com_ls = list(com_df.company)

# Data 2017

富比士2000資料

In [48]:
f_data = pd.read_csv("./Forbes/Forbes_2017.csv")

In [49]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue
14,Samsung Electronics,254.303,173.995,19.3207,217.068,15,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Oh-Hyun Kwon,http://www.forbes.com/companies/samsung-electr...,0.089008,0.111042
53,Intel,170.350,59.387,10.3160,113.327,54,Information Technology,Semiconductors,North America,United States,California,California,Brian M. Krzanich,http://www.forbes.com/companies/intel/,0.091029,0.173708
126,Taiwan Semiconductor,161.734,29.392,10.3640,58.533,127,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Te Yin Liu,http://www.forbes.com/companies/taiwan-semicon...,0.177063,0.352613
169,Qualcomm,83.187,23.778,4.8890,52.366,170,Information Technology,Semiconductors,North America,United States,California,California,Steven M. Mollenkopf,http://www.forbes.com/companies/qualcomm/,0.093362,0.205610
348,SK Hynix,31.958,14.823,2.5459,26.673,349,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Sung-Wook Park,http://www.forbes.com/companies/sk-hynix/,0.095449,0.171753
386,Texas Instruments,80.501,13.370,3.4170,16.431,387,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard K. Templeton,http://www.forbes.com/companies/texas-instrume...,0.207961,0.255572
481,Micron Technology,31.552,14.733,0.6890,32.355,482,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,Sanjay Mehrotra,http://www.forbes.com/companies/micron-technol...,0.021295,0.046766
498,Applied Materials,42.038,11.846,2.1380,15.244,499,Information Technology,Semiconductors,North America,United States,California,California,Gary E. Dickerson,http://www.forbes.com/companies/applied-materi...,0.140252,0.180483
530,ASML Holding,56.800,7.516,1.7223,19.742,531,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter T. F. M. Wennink,http://www.forbes.com/companies/asml-holding/,0.087240,0.229151
591,Broadcom,87.597,15.608,-1.8770,49.617,592,Information Technology,Semiconductors,Asia,Singapore,Singapore,NaN,Hock E. Tan,http://www.forbes.com/companies/broadcom/,-0.037830,-0.120259


In [50]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [51]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [58]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
0,Samsung Electronics,678,6.519147
1,Qualcomm,655,6.484635
3,Intel,403,5.998937


In [52]:
# pbc 0.1,0.2
comp_ls = ["Qualcomm","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB","Shenzhen Shenglu IoT Communication Technology"]

In [53]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [59]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
1,Qualcomm,655,6.484635
2,Shenzhen Shenglu IoT Communication Technology,622,6.432940
3,Intel,403,5.998937
4,Telefonaktiebolaget LM Ericsson AB,395,5.978886
5,Huawei Technologies,291,5.673323
6,IBM,249,5.517453


In [54]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.14346946548878017
ari:  0.2705943901163025


## 使用Forbes做join

In [24]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [25]:
# pbc 0.1,0.2
comp_ls = ["Qualcomm","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB","Shenzhen Shenglu IoT Communication Technology"]

In [26]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [27]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.08200321388291647
ari:  0.1292559483889141
